# Data Analysis

In order to test an SVM with HOG features in practice, we decide to use the LFW dataset that is built-in sklearn. To start, we imported the dataset:

In [ ]:
from typing import Callable

import cv2
import numpy as np
import dlib
from PIL import Image, ImageFilter
from sklearn import datasets
from matplotlib import pyplot as plt
import sys
sys.path.append('../test/')
import ImageModification as imod

def get_faces():
    return datasets.fetch_lfw_people(color=True, min_faces_per_person=100, resize=1)

Afterwards, we created the function `count_detected_faces_hog` that counts the number of faces that the SVM classifier with HOG features from dlib detected. The parameter `image_mod_fn` is a function that modifies the image before it is fed to the SVM classifier.

In [ ]:
def count_detected_faces_hog(image_mod_fn: Callable[[np.ndarray], np.ndarray]):
    faces = get_faces()
    detector = dlib.get_frontal_face_detector()
    counter_faces = 0

    for possibleFace in faces.images:
        scaled_rgb_image = (possibleFace * 255).astype(np.uint8)

        modifiedImage = image_mod_fn(scaled_rgb_image)

        detected_faces = detector(modifiedImage)

        if len(detected_faces) > 0:
            counter_faces += 1

    return float(counter_faces) / float(len(faces.images))

Then we used several modifications and plotted the success rate of detecting the faces in 1288 images:

In [ ]:
def image_modification_plot(count_detected_faces: Callable[[Callable[[np.ndarray], np.ndarray]], float], title: str):
    categories = ["Unmodified", "Blur", "Rotate 20°", "Rotate 90°", "Flip Horizontally", "Change to Grayscale"]
    values = [count_detected_faces(imod.identity), count_detected_faces(imod.apply_blur),
              count_detected_faces(imod.create_rotate_image(20)),
              count_detected_faces(imod.create_rotate_image(90)), count_detected_faces(imod.flip_image_horizontally),
              count_detected_faces(imod.change_to_grayscale)]

    # Create bar chart
    plt.figure(figsize=(8, 6))
    plt.bar(categories, values, color='skyblue')
    plt.ylim(0, 1.0)

    # Title and labels
    plt.title(title)
    plt.xlabel('Modification Operations')
    plt.ylabel('Proportion detected')

    # Show plot
    plt.xticks(rotation=45)  # Rotate x-axis labels for better visibility
    plt.tight_layout()
    plt.show()
    
image_modification_plot(count_detected_faces_hog, "HOG modifications")

The bar chart shows that HOG is not susceptible to a horizontal flip, a conversion to grayscale, and a small rotation, but it is susceptible to blur and a 90° rotation.